## Importação das Bibiliotecas

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

## Criando uma SparkSession

In [2]:
spark = SparkSession.builder.appName("LerDadosEnem").getOrCreate()


## Processo de Leitura do CSV de Participantes


In [3]:
arqEnem = 'E:\Estudos\SQL\Datasets\microdados_enem_2022\DADOS\MICRODADOS_ENEM_2022.csv'

dfDadosGerais = spark.read.csv(arqEnem, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')


In [9]:
dfProvas = dfDadosGerais.select(dfDadosGerais.columns[12:19])

In [11]:
dfProvas.count()

3476105

In [13]:
dfPreencher = dfProvas.fillna(-2)  # Preenche com -2(sem ref), por exemplo


# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER Usando JDBC

In [14]:
server_name = "jdbc:sqlserver://NOTEBOOK-LEO:1433"
database_name = "Enem"
table_name = "Provas"
username = "bi_leonardobo"
password = "0000"
url = server_name + ";" + "databaseName=" + database_name + ";"
print(url)


jdbc:sqlserver://NOTEBOOK-LEO:1433;databaseName=Enem;


In [15]:
# Nome da nova tabela que será criada
nomeTabela = table_name

In [16]:
#Conectando via jdbc
#No método option, foi necessário passar os orgumentos ssl e trustservercertificate como true, por se tratar de um banco local sem SSL.
try:
  jdbcDF = spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()
  
  jdbcDF.show(40)
  print(jdbcDF.count())
except ValueError as error :
    print("Connector write failed", error)


+----------+-------+-------+-----------+-------------+------------+--------------+----------+----------+----------+------+--------+
|CO_POSICAO|SG_AREA|CO_ITEM|TX_GABARITO|CO_HABILIDADE|IN_ITEM_ABAN|TX_MOTIVO_ABAN|NU_PARAM_A|NU_PARAM_B|NU_PARAM_C|TX_COR|CO_PROVA|
+----------+-------+-------+-----------+-------------+------------+--------------+----------+----------+----------+------+--------+
+----------+-------+-------+-----------+-------------+------------+--------------+----------+----------+----------+------+--------+

0


In [17]:
# Configurações do banco de dados em ODBC para dropar
server_name = "NOTEBOOK-LEO\\SQLEXPRESS"
 
# Conecte-se ao banco de dados usando pyodbc
conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server_name};Database={database_name};UID={username};PWD={password}")

# Crie um cursor
cursor = conn.cursor()

# Execute a operação DROP TABLE na tabela 'Participantes'
truncate_query = f"DROP TABLE {table_name}"
cursor.execute(truncate_query)

# Commit as alterações no banco de dados
conn.commit()

In [18]:
# Escreva os novos dados em 'Participantes' usando a conexão JDBC
dfPreencher.write \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("SaveMode", "overWrite").save()
  


In [19]:
print(f'Tabela {nomeTabela} criada com sucesso no banco de dados.')


Tabela Provas criada com sucesso no banco de dados.
